In [1]:
# Block 1: Importing Libraries
import tarfile
import resampy
import pandas as pd
import librosa
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam


from concurrent.futures import ThreadPoolExecutor

In [2]:
# Block 2: Extracting Data
import shutil
import tarfile
import os

def extract_tar(tar_file, target_dir):
    if os.path.exists(target_dir):
        user_input = input(f"The directory '{target_dir}' already exists. Do you want to skip extraction? (y/n): ")
        if user_input.lower() == 'y':
            print(f"Skipping extraction of {tar_file}.")
            return
        else:
            print(f"Overwriting the existing directory '{target_dir}'.")
            shutil.rmtree(target_dir)

    with tarfile.open(tar_file, 'r') as tar:
        tar.extractall(target_dir)

    # Remove residue "._" hidden files from the inner folder
    inner_folder = os.path.join(target_dir, os.path.splitext(os.path.basename(tar_file))[0])
    for root, dirs, files in os.walk(inner_folder):
        for file in files:
            if file.startswith("._"):
                os.remove(os.path.join(root, file))

extract_tar('train_mp3s.tar', 'train_mp3s')
extract_tar('test_mp3s.tar', 'test_mp3s')
train_labels = np.loadtxt('train_label.txt', dtype=int)

Skipping extraction of train_mp3s.tar.
Skipping extraction of test_mp3s.tar.


In [3]:
# Block 3: Preprocessing Functions
def preprocess_audio(file_path):
    try:
        audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
        print(f"Loaded audio file: {file_path}")
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_scaled = np.mean(mfccs.T, axis=0)
        print(f"Extracted MFCCs: {mfccs_scaled.shape}")
        return mfccs_scaled
    except Exception as e:
        print(f"Error processing file: {file_path}")
        print(f"Error message: {str(e)}")
        return None

def process_file(file_path):
    print(f"Processing file: {file_path}")
    mfccs = preprocess_audio(file_path)
    return mfccs

def prepare_data(directory):
    file_paths = [os.path.join(directory, f"{i}.mp3") for i in range(len(os.listdir(directory)))]
    
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(process_file, file_paths))
    
    features = [mfccs for mfccs in results if mfccs is not None]
    print(f"Processed {len(features)} audio files")
    return np.array(features)

In [4]:
# Block 4: Preparing Data
train_features = prepare_data('train_mp3s/train_mp3s')
print(f"Train features shape: {train_features.shape}")

test_features = prepare_data('test_mp3s/test_mp3s')
print(f"Test features shape: {test_features.shape}")

train_labels = np.array([int(label) for label in train_labels])
print(f"Train labels shape: {train_labels.shape}")

print(f"Number of training features: {len(train_features)}")
print(f"Number of training labels: {len(train_labels)}")
print(f"Number of test features: {len(test_features)}")

if len(train_features) == 0:
    print("No training features available. Please check the data.")

Processing file: train_mp3s/train_mp3s\0.mp3
Processing file: train_mp3s/train_mp3s\1.mp3
Processing file: train_mp3s/train_mp3s\2.mp3
Processing file: train_mp3s/train_mp3s\3.mp3
Processing file: train_mp3s/train_mp3s\4.mp3
Processing file: train_mp3s/train_mp3s\5.mp3
Processing file: train_mp3s/train_mp3s\6.mp3
Processing file: train_mp3s/train_mp3s\7.mp3
Processing file: train_mp3s/train_mp3s\8.mp3
Processing file: train_mp3s/train_mp3s\9.mp3
Processing file: train_mp3s/train_mp3s\10.mp3
Processing file: train_mp3s/train_mp3s\11.mp3
Processing file: train_mp3s/train_mp3s\12.mp3
Processing file: train_mp3s/train_mp3s\13.mp3
Processing file: train_mp3s/train_mp3s\14.mp3
Processing file: train_mp3s/train_mp3s\15.mp3
Processing file: train_mp3s/train_mp3s\16.mp3
Processing file: train_mp3s/train_mp3s\17.mp3
Processing file: train_mp3s/train_mp3s\18.mp3
Processing file: train_mp3s/train_mp3s\19.mp3
Processing file: train_mp3s/train_mp3s\20.mp3
Processing file: train_mp3s/train_mp3s\21.mp

In [ ]:
# Block 5: Model Training and Prediction

if len(train_features) > 0:
    train_data, val_data, train_labels, val_labels = train_test_split(train_features, train_labels, test_size=0.3, random_state=42)
    
    model = Sequential([
        Dense(128, activation='relu', input_shape=(40,)),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(4, activation='softmax')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=120, batch_size=32)
    
    predictions = model.predict(test_features)
    predicted_labels = np.argmax(predictions, axis=1)
    
    submission = pd.DataFrame({'id': range(len(predicted_labels)), 'category': predicted_labels})
    submission.to_csv('submission.csv', index=False)
else:
    print("No training features available. Please check the data.")

Epoch 1/120


c:\Users\mojii\anaconda3\envs\ML\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


260/260 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3031 - loss: 16.0292 - val_accuracy: 0.3671 - val_loss: 1.2899
Epoch 2/120
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3565 - loss: 1.5759 - val_accuracy: 0.3702 - val_loss: 1.2760
Epoch 3/120
260/260 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3611 - loss: 1.3563 - val_accuracy: 0.3746 - val_loss: 1.2518
Epoch 4/120
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3826 - loss: 1.3000 - val_accuracy: 0.3836 - val_loss: 1.2026
Epoch 5/120
260/260 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3952 - loss: 1.2471 - val_accuracy: 0.4638 - val_loss: 1.1470
Epoch 6/120
260/260 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4153 - loss: 1.1917 - val_accuracy: 0.5230 - val_loss: 1.0871
Epoch 7/120
260/260 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4545 - loss: 1.1520 - val_accuracy: 0.5766 - val_loss: 1.0234
Epoch 8/120
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4862 - loss: 1.1027 - val_accuracy: 0.585